In [1]:
import sys
import os

sys.path.insert(0, os.path.abspath("."))
sys.path.append(os.path.abspath("../../"))

In [2]:
import numpy as np
np.set_printoptions(linewidth=np.inf, precision=4, suppress=True, threshold=sys.maxsize)
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.graph_objects as go
import functools
import scipy

In [3]:
from desc import set_device

num_device = 2
# _set_cpu_count(num_device)
set_device("gpu", num_device=num_device)

In [4]:
import desc

from desc.basis import *
from desc.backend import *
from desc.compute import *
from desc.coils import *
from desc.equilibrium import *
from desc.examples import *
from desc.grid import *
from desc.geometry import *

from desc.objectives import *
from desc.objectives.objective_funs import *
from desc.objectives.getters import *
from desc.objectives.normalization import compute_scaling_factors
from desc.objectives.utils import *
from desc.optimize._constraint_wrappers import *

from desc.transform import Transform
from desc.plotting import *
from desc.optimize import *
from desc.perturbations import *
from desc.profiles import *
from desc.compat import *
from desc.utils import *

from desc.__main__ import main
from desc.vmec_utils import vmec_boundary_subspace
from desc.input_reader import InputReader
from desc.continuation import solve_continuation_automatic

DESC version 0.13.0+1543.g3edf125e0,using JAX backend, jax version=0.5.0, jaxlib version=0.5.0, dtype=float64
Using 2 device:
	 Device 0: NVIDIA A100-SXM4-40GB (id=0) with 40.00 GB available memory
	 Device 1: NVIDIA A100-SXM4-40GB (id=1) with 40.00 GB available memory


In [12]:
from mpi4py import MPI
import jax
import jax.numpy as jnp
import mpi4jax.mpi4jax as mpi4jax

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()
print(comm, rank, size)

@jax.jit
def foo(arr):
   arr = arr + rank
   arr_sum, _ = mpi4jax.allreduce(arr, op=MPI.SUM, comm=comm)
   return arr_sum

a = jnp.zeros((3, 3))
result = foo(a)

if rank == 0:
   print(result)

<mpi4py.MPI.Intracomm object at 0x14f333fff510> 0 1
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


In [ ]:
from mpi4py.futures import MPIPoolExecutor
from mpi4py import MPI
import jax
import jax.numpy as jnp
import mpi4jax

def mpi_worker(rank):
    comm = MPI.COMM_WORLD
    size = comm.Get_size()
    print(comm, rank, size)

    @jax.jit
    def foo(arr):
        arr = arr + rank
        arr_sum, _ = mpi4jax.allreduce(arr, op=MPI.SUM, comm=comm)
        return arr_sum

    a = jnp.zeros((3, 3))
    result = foo(a)
    return result if rank == 0 else None

def main():
    with MPIPoolExecutor() as executor:
        ranks = list(range(executor._max_workers))
        results = list(executor.map(mpi_worker, ranks))
        
        for res in results:
            if res is not None:
                print(res)

main()

--------------------------------------------------------------------------
There are not enough slots available in the system to satisfy the 1
slots that were requested by the application:

  /home/ye2698/.conda/envs/desc-mpi/bin/python

Either request fewer slots for your application, or make more slots
available for use.

A "slot" is the Open MPI term for an allocatable unit where we can
launch a process.  The number of slots available are defined by the
environment in which Open MPI processes are run:

  1. Hostfile, via "slots=N" clauses (N defaults to number of
     processor cores if not provided)
  2. The --host command line parameter, via a ":N" suffix on the
     hostname (N defaults to 1 if not provided)
  3. Resource manager (e.g., SLURM, PBS/Torque, LSF, etc.)
  4. If none of a hostfile, the --host command line parameter, or an
     RM is present, Open MPI defaults to the number of processor cores

In all the above cases, if you want Open MPI to default to the number
of har